In [1]:
cd C:/Users/PC/Jeehyun/ECE6780/Data/vgg/

C:\Users\PC\Jeehyun\ECE6780\Data\vgg


In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, Activation
from keras import applications
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.applications.inception_v3 import InceptionV3
from keras.models import model_from_json

Using TensorFlow backend.


In [3]:
img_width, img_height = 224, 224
def gen_name(mag_fact):
    train_dir = 'breast_' + mag_fact + '_split/train'
    val_dir = 'breast_' + mag_fact + '_split/val'
    model_name = 'bottleneck_fc_model_' + mag_fact + '.h5'
    return train_dir, val_dir, model_name

In [4]:
def fine_train():
    model_new_name = 'TL_model_' + mag_fact
    top_model_weights_path = model_new_name + '.h5'
    model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
    print('Model loaded.')

    top_model = Sequential()
    top_model.add(Flatten(input_shape=model.output_shape[1:]))
    top_model.add(Dense(256))
    top_model.add(Activation('relu'))
    top_model.add(Dropout(0.25))
    top_model.add(Dense(1))
    top_model.add(Activation('sigmoid'))
    top_model.load_weights(top_model_weights_path)

    newmodel = Model(input= model.input, output= top_model(model.output))

    for layer in newmodel.layers[:15]:
        layer.trainable = False
    
    newmodel.compile(loss='binary_crossentropy',
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  metrics=['accuracy'])


    train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')
    validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

    newmodel.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        epochs=epochs,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)
    
    fine_features_train = newmodel.predict_generator(train_generator, nb_train_samples//batch_size)
    np.save(open('fine_features_train' + mag_fact +'.npy', 'wb'), fine_features_train)

    fine_features_val = newmodel.predict_generator(validation_generator, nb_train_samples//batch_size)
    np.save(open('fine_features_val' + mag_fact +'.npy', 'wb'), fine_features_val)

    model_new_name = 'tuned_TL_' + mag_fact
    model_json = newmodel.to_json()

    with open(model_new_name + '.json', "w") as json_file:
        json_file.write(model_json)
        newmodel.save_weights(model_new_name + '.h5')

In [5]:
mag_fact = '40'
b_train, m_train, b_test, m_test, b_val, m_val = 500, 1170, 100, 137, 137, 174

nb_train_samples =  b_train + m_train
nb_validation_samples =  b_val + m_val

train_data_dir, validation_data_dir, model_name = gen_name(mag_fact)

batch_size = 1
epochs = 20
fine_train()

Model loaded.


C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  app.launch_new_instance()


Found 1670 images belonging to 2 classes.
Found 311 images belonging to 2 classes.


C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=20, validation_data=<keras.pre..., steps_per_epoch=1670, validation_steps=311)`


Epoch 1/20
1670/1670 [==============================] - 814s 487ms/step - loss: 0.5654 - accuracy: 0.7263 - val_loss: 2.6413 - val_accuracy: 0.5723
Epoch 2/20
1670/1670 [==============================] - 862s 516ms/step - loss: 0.4790 - accuracy: 0.7832 - val_loss: 0.5219 - val_accuracy: 0.6527
Epoch 3/20
1670/1670 [==============================] - 761s 456ms/step - loss: 0.4120 - accuracy: 0.8228 - val_loss: 0.0747 - val_accuracy: 0.7621
Epoch 4/20
1670/1670 [==============================] - 696s 417ms/step - loss: 0.3600 - accuracy: 0.8509 - val_loss: 0.2391 - val_accuracy: 0.8875
Epoch 5/20
1670/1670 [==============================] - 697s 417ms/step - loss: 0.3304 - accuracy: 0.8653 - val_loss: 3.9545e-07 - val_accuracy: 0.8360
Epoch 6/20
1670/1670 [==============================] - 697s 417ms/step - loss: 0.3089 - accuracy: 0.8713 - val_loss: 0.0226 - val_accuracy: 0.8810
Epoch 7/20
1670/1670 [==============================] - 697s 417ms/step - loss: 0.2576 - accuracy: 0.8970 - 

In [7]:
mag_fact = '100'
b_train, m_train, b_test, m_test, b_val, m_val = 515, 1237, 100, 145, 135, 188

nb_train_samples =  b_train + m_train
nb_validation_samples =  b_val + m_val

train_data_dir, validation_data_dir, model_name = gen_name(mag_fact)

batch_size = 1
epochs = 10
fine_train()

Model loaded.
Found 1752 images belonging to 2 classes.


C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  app.launch_new_instance()


Found 323 images belonging to 2 classes.


C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=10, validation_data=<keras.pre..., steps_per_epoch=1752, validation_steps=323)`


Epoch 1/10
1752/1752 [==============================] - 771s 440ms/step - loss: 0.5901 - accuracy: 0.7072 - val_loss: 1.1405 - val_accuracy: 0.5820
Epoch 2/10
1752/1752 [==============================] - 728s 416ms/step - loss: 0.5285 - accuracy: 0.7146 - val_loss: 1.3731 - val_accuracy: 0.8111
Epoch 3/10
1752/1752 [==============================] - 729s 416ms/step - loss: 0.4412 - accuracy: 0.7803 - val_loss: 0.9541 - val_accuracy: 0.6533
Epoch 4/10
1752/1752 [==============================] - 729s 416ms/step - loss: 0.3633 - accuracy: 0.8402 - val_loss: 0.0460 - val_accuracy: 0.7461
Epoch 5/10
1752/1752 [==============================] - 728s 416ms/step - loss: 0.3224 - accuracy: 0.8556 - val_loss: 0.4650 - val_accuracy: 0.8545
Epoch 6/10
1752/1752 [==============================] - 729s 416ms/step - loss: 0.3042 - accuracy: 0.8710 - val_loss: 1.5785e-05 - val_accuracy: 0.8916
Epoch 7/10
1752/1752 [==============================] - 730s 416ms/step - loss: 0.2524 - accuracy: 0.9081 - 

In [8]:
mag_fact = '200'
b_train, m_train, b_test, m_test, b_val, m_val = 498, 1190, 100, 139, 137, 178

nb_train_samples =  b_train + m_train
nb_validation_samples =  b_val + m_val

train_data_dir, validation_data_dir, model_name = gen_name(mag_fact)

batch_size = 1
epochs = 10
fine_train()

Model loaded.
Found 1688 images belonging to 2 classes.


C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  app.launch_new_instance()


Found 315 images belonging to 2 classes.


C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=10, validation_data=<keras.pre..., steps_per_epoch=1688, validation_steps=315)`


Epoch 1/10
1688/1688 [==============================] - 753s 446ms/step - loss: 0.5818 - accuracy: 0.7079 - val_loss: 0.3368 - val_accuracy: 0.5651
Epoch 2/10
1688/1688 [==============================] - 703s 416ms/step - loss: 0.5307 - accuracy: 0.7168 - val_loss: 1.2432 - val_accuracy: 0.6857
Epoch 3/10
1688/1688 [==============================] - 703s 416ms/step - loss: 0.4550 - accuracy: 0.7790 - val_loss: 0.7935 - val_accuracy: 0.8476
Epoch 4/10
1688/1688 [==============================] - 702s 416ms/step - loss: 0.4025 - accuracy: 0.8122 - val_loss: 0.5595 - val_accuracy: 0.7905
Epoch 5/10
1688/1688 [==============================] - 703s 416ms/step - loss: 0.3322 - accuracy: 0.8590 - val_loss: 0.0331 - val_accuracy: 0.7937
Epoch 6/10
1688/1688 [==============================] - 705s 418ms/step - loss: 0.3084 - accuracy: 0.8596 - val_loss: 3.7536 - val_accuracy: 0.7746
Epoch 7/10
1688/1688 [==============================] - 703s 417ms/step - loss: 0.2656 - accuracy: 0.8803 - val_

KeyboardInterrupt: 

In [11]:
mag_fact = '400'
b_train, m_train, b_test, m_test, b_val, m_val = 470, 1032, 100, 124, 140, 147

nb_train_samples =  b_train + m_train
nb_validation_samples =  b_val + m_val

train_data_dir, validation_data_dir, model_name = gen_name(mag_fact)

batch_size = 1
epochs = 10
fine_train()

Model loaded.
Found 1502 images belonging to 2 classes.
Found 287 images belonging to 2 classes.

C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  app.launch_new_instance()


C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=10, validation_data=<keras.pre..., steps_per_epoch=1502, validation_steps=287)`


Epoch 1/10
1502/1502 [==============================] - 630s 420ms/step - loss: 0.6212 - accuracy: 0.6877 - val_loss: 1.1308 - val_accuracy: 0.5122
Epoch 2/10
1502/1502 [==============================] - 714s 476ms/step - loss: 0.6160 - accuracy: 0.6897 - val_loss: 0.3270 - val_accuracy: 0.5122
Epoch 3/10
1502/1502 [==============================] - 654s 435ms/step - loss: 0.5634 - accuracy: 0.7270 - val_loss: 0.2836 - val_accuracy: 0.7735
Epoch 4/10
1502/1502 [==============================] - 627s 417ms/step - loss: 0.4823 - accuracy: 0.7830 - val_loss: 1.5560 - val_accuracy: 0.5122
Epoch 5/10
1502/1502 [==============================] - 627s 417ms/step - loss: 0.4571 - accuracy: 0.7929 - val_loss: 0.1561 - val_accuracy: 0.6307
Epoch 6/10
1502/1502 [==============================] - 627s 417ms/step - loss: 0.3755 - accuracy: 0.8469 - val_loss: 0.0884 - val_accuracy: 0.6341
Epoch 7/10
1502/1502 [==============================] - 628s 418ms/step - loss: 0.3728 - accuracy: 0.8415 - val_

In [12]:
def mag_conn(mag):
    train_dir = 'breast_' + mag + '_split/train'
    val_dir = 'breast_' + mag + '_split/val'
    test_dir = 'breast_' + mag + '_split/test'

    if mag == '40':
        b_train, m_train, b_test, m_test, b_val, m_val = 500, 1170, 100, 137, 137, 174
    if mag == '100':
        b_train, m_train, b_test, m_test, b_val, m_val = 515, 1237, 100, 145, 135, 188
    if mag == '200':
        b_train, m_train, b_test, m_test, b_val, m_val = 498, 1190, 100, 139, 137, 178
    if mag == '400':
        b_train, m_train, b_test, m_test, b_val, m_val = 470, 1032, 100, 124, 140, 147        
    
    n_train =  b_train + m_train
    n_val =  b_val + m_val
    n_test =  b_test + m_test

    train_labels = np.array([0] * b_train + [1] * m_train)
    val_labels = np.array([0] * b_val + [1] * m_val)
    test_labels = np.array([0] * b_test + [1] * m_test)
    return train_dir, val_dir, test_dir, n_train, n_val, n_test, train_labels, val_labels, test_labels

In [13]:
def VGG_feat(mag):
    vgg_train = np.load('fine_bottleneck_feat_train_' + mag +'.npy')
    vgg_val = np.load('fine_bottleneck_feat_val_' + mag +'.npy')
    vgg_test = np.load('fine_bottleneck_feat_test_' + mag +'.npy')
    return vgg_train, vgg_val, vgg_test

In [21]:
def test_breakhis(mag):
    train_dir, val_dir, test_dir, nb_train, nb_val, nb_test, train_labels, val_labels, test_labels = mag_conn(mag)
    VGG_train, VGG_val, VGG_test = VGG_feat(mag)
    
    test_datagen = ImageDataGenerator(rescale=1. / 255)
    train_generator = test_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')
    val_generator = test_datagen.flow_from_directory(
        val_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')
        
    model_name = 'tuned_TL_' + mag
    # load json and create model
    json_file = open(model_name + '.json', 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    # load weights into new model
    model.load_weights(model_name + '.h5')
    print("Loaded model from disk")
    # evaluate loaded model on test data
    model.compile(optimizer='sgd',loss='binary_crossentropy', metrics=['accuracy'])
    
    
    train_score = model.evaluate_generator(train_generator)
    val_score = model.evaluate_generator(val_generator)
    test_score = model.evaluate_generator(test_generator)
    print('\n', 'BreakHis Train accuracy:', train_score[1])
    print('\n', 'BreakHis Validation accuracy:', val_score[1])
    print('\n', 'BreakHis Test accuracy:', test_score[1])


In [19]:
mag_fact = '100'
b_train, m_train, b_test, m_test, b_val, m_val = 470, 1032, 100, 124, 140, 147

nb_train_samples =  b_train + m_train
nb_validation_samples =  b_val + m_val

train_data_dir, validation_data_dir, model_name = gen_name(mag_fact)

batch_size = 1
epochs = 10
fine_train()
test_breakhis('100')

mag_fact = '400'
b_train, m_train, b_test, m_test, b_val, m_val = 470, 1032, 100, 124, 140, 147

nb_train_samples =  b_train + m_train
nb_validation_samples =  b_val + m_val

train_data_dir, validation_data_dir, model_name = gen_name(mag_fact)

batch_size = 1
epochs = 10
fine_train()
test_breakhis('400')

Model loaded.
Found 1752 images belonging to 2 classes.


C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  app.launch_new_instance()


Found 323 images belonging to 2 classes.


C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=10, validation_data=<keras.pre..., steps_per_epoch=1502, validation_steps=287)`


Epoch 1/10
1502/1502 [==============================] - 629s 419ms/step - loss: 0.5746 - accuracy: 0.7150 - val_loss: 0.8920 - val_accuracy: 0.5819
Epoch 2/10
1502/1502 [==============================] - 628s 418ms/step - loss: 0.5127 - accuracy: 0.7204 - val_loss: 0.7327 - val_accuracy: 0.6655
Epoch 3/10
1502/1502 [==============================] - 627s 418ms/step - loss: 0.4404 - accuracy: 0.7876 - val_loss: 0.3171 - val_accuracy: 0.7735
Epoch 4/10
1502/1502 [==============================] - 627s 418ms/step - loss: 0.3848 - accuracy: 0.8296 - val_loss: 6.0305e-04 - val_accuracy: 0.8432
Epoch 5/10
1502/1502 [==============================] - 627s 417ms/step - loss: 0.3444 - accuracy: 0.8515 - val_loss: 0.2625 - val_accuracy: 0.7700
Epoch 6/10
1502/1502 [==============================] - 626s 417ms/step - loss: 0.2988 - accuracy: 0.8675 - val_loss: 0.2165 - val_accuracy: 0.8606
Epoch 7/10
1502/1502 [==============================] - 627s 417ms/step - loss: 0.2633 - accuracy: 0.8768 - 

C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  app.launch_new_instance()
C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=10, validation_data=<keras.pre..., steps_per_epoch=1502, validation_steps=287)`


Epoch 1/10
1502/1502 [==============================] - 627s 417ms/step - loss: 0.6217 - accuracy: 0.6871 - val_loss: 0.3830 - val_accuracy: 0.5122
Epoch 2/10
1502/1502 [==============================] - 627s 417ms/step - loss: 0.6208 - accuracy: 0.6871 - val_loss: 1.0726 - val_accuracy: 0.5122
Epoch 3/10
1502/1502 [==============================] - 626s 417ms/step - loss: 0.5934 - accuracy: 0.7084 - val_loss: 1.1950 - val_accuracy: 0.5122
Epoch 4/10
1502/1502 [==============================] - 626s 417ms/step - loss: 0.5380 - accuracy: 0.7490 - val_loss: 2.1269 - val_accuracy: 0.5645
Epoch 5/10
1502/1502 [==============================] - 627s 418ms/step - loss: 0.4560 - accuracy: 0.8003 - val_loss: 0.1885 - val_accuracy: 0.6411
Epoch 6/10
1502/1502 [==============================] - 626s 417ms/step - loss: 0.3977 - accuracy: 0.8349 - val_loss: 0.3106 - val_accuracy: 0.8606
Epoch 7/10
1502/1502 [==============================] - 626s 417ms/step - loss: 0.3780 - accuracy: 0.8349 - val_

In [22]:
test_breakhis('40')
test_breakhis('100')
test_breakhis('200')
test_breakhis('400')

Found 1670 images belonging to 2 classes.
Found 311 images belonging to 2 classes.
Found 237 images belonging to 2 classes.
Loaded model from disk

 BreakHis Train accuracy: 0.9904191493988037

 BreakHis Validation accuracy: 0.9742765426635742

 BreakHis Test accuracy: 0.9535865187644958
Found 1752 images belonging to 2 classes.
Found 323 images belonging to 2 classes.
Found 245 images belonging to 2 classes.
Loaded model from disk

 BreakHis Train accuracy: 0.943493127822876

 BreakHis Validation accuracy: 0.9164086580276489

 BreakHis Test accuracy: 0.9387755393981934
Found 1688 images belonging to 2 classes.
Found 315 images belonging to 2 classes.
Found 239 images belonging to 2 classes.
Loaded model from disk

 BreakHis Train accuracy: 0.9911137223243713

 BreakHis Validation accuracy: 0.9555555582046509

 BreakHis Test accuracy: 0.9205020666122437
Found 1502 images belonging to 2 classes.
Found 287 images belonging to 2 classes.
Found 224 images belonging to 2 classes.
Loaded mod